In [1]:
SCHOOL="middlebury"
SUBJECT="opinion"
START_YEAR=2005
FINAL_YEAR=2022
DATA_DIR="data"
OUTPUT_DIR="output"

In [2]:
%pwd

'/Users/angela/diversity-colleges/notebooks'

In [3]:
cd ..

/Users/angela/diversity-colleges


In [4]:
import sys
sys.path.append('src')
import ouraws
import ourgraphs
import textutil

In [5]:
S3OBJECT_KEY = f"{DATA_DIR}/{SCHOOL}-{SUBJECT}-SNAPSHOT.parquet"
df = ouraws.getFromS3(S3OBJECT_KEY)

In [6]:
df.shape

(2984, 4)

In [7]:
df.head(2)

,url,body,year,month
0,https://www.middleburycampus.com/article/2022/...,"\nIn a couple of weeks, the last class who has...",2022,5
1,https://www.middleburycampus.com/article/2023/...,\nDeep breath. I turn on my phone and go to op...,2023,1


In [8]:
import requests
import os

BIPARTISAN_API_KEY = os.environ.get("BIPARTISAN_API")

BIPARTISAN_URL = "https://api.thebipartisanpress.com/api/endpoints/beta/robert"

In [9]:
df2022 = df[df['year'] == 2022]

articles_list = df2022['body'].to_list()
for article in articles_list:
    payload = {"API": BIPARTISAN_API_KEY, "Text": article.encode("utf-8")}
    response = requests.post(BIPARTISAN_URL, data=payload)
    print(response.text)
    break

-8.093837


In [10]:
df['year'].unique()

array([2022, 2023, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013,
       2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005, 2003, 2002, 2001])

In [12]:
import pandas as pd

RESULTS_FILE = f"{DATA_DIR}/{SCHOOL}-POLARITY.parquet"

In [13]:
import asyncio

results = []

for year in range(2009, 2023):
    df2 = df[df['year'] == year]
    print(f"Year: {year} ... {df2.shape[0]}")

    articles_list = df2['body'].to_list()
    value_sum = 0.0
    article_count = 0
    for article in articles_list:
        payload = {"API": BIPARTISAN_API_KEY, "Text": article.encode("utf-8")}
        response = requests.post(BIPARTISAN_URL, data=payload)
        try:
            value_sum += float(response.text)
            article_count += 1
            # print(f"{article_count}", end=".")
        except:
            pass  # this is a non-number
        
    results.append({
        'year' : year,
        'article_count' : article_count,
        'polarity_sum' : value_sum,
        'polarity_avg' : float(value_sum) / article_count
    })

    print(f"{year}\t{article_count}\t{value_sum}\t{value_sum/article_count}")
# for each year, get all articles' bodies & send to bipartisan press API 
#    and sum up the values & output the average for each year

Year: 2009 ... 131
2009	128	-20.810897999999977	-0.16258514062499982
Year: 2010 ... 165
2010	162	-207.32845200000008	-1.2798052592592597
Year: 2011 ... 107
2011	106	-139.906407	-1.3198717641509434
Year: 2012 ... 103
2012	101	-380.83983500000005	-3.770691435643565
Year: 2013 ... 213
2013	208	-849.4407909999995	-4.083849956730767
Year: 2014 ... 175
2014	171	-510.7054840000001	-2.9865817777777783
Year: 2015 ... 174
2015	161	-490.61999000000003	-3.047329130434783
Year: 2016 ... 59
2016	59	-420.466812	-7.12655613559322
Year: 2017 ... 126
2017	114	-749.7635340000002	-6.576873105263159
Year: 2018 ... 153
2018	150	-883.6915749999993	-5.8912771666666615
Year: 2019 ... 173
2019	172	-855.2545630000001	-4.97241025
Year: 2020 ... 155
2020	155	-1197.4211870000004	-7.725297980645164
Year: 2021 ... 78
2021	78	-542.8265010000001	-6.9593141153846165
Year: 2022 ... 76
2022	75	-500.7877909999999	-6.677170546666665


In [14]:
results_df = pd.DataFrame.from_records(results)
results_df.to_parquet(RESULTS_FILE)